<a href="https://colab.research.google.com/github/jungeKatz/AItweetBot/blob/main/tweet_simulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time


In [51]:
with open('/content/drive/MyDrive/nlp/extract.txt', 'r') as file:
    text = file.read().lower()
print('text length', len(text))

text length 4576053


In [52]:
print(text[:250])


painful to see the devastating impact of extreme weather in western  with the unprecedented loss of life &amp; destructio n of property. may the world unite in preventing such climatic upheavals by investing in regenerating the planet. – sg what happ


In [53]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')


499 unique characters


In [54]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [55]:
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab), mask_token=None)


In [56]:
ids = ids_from_chars(chars)
ids


<tf.RaggedTensor [[37, 38, 39, 40, 41, 42, 43], [60, 61, 62]]>

In [57]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)


In [58]:
chars = chars_from_ids(ids)
chars


<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [59]:
tf.strings.reduce_join(chars, axis=-1).numpy()


array([b'abcdefg', b'xyz'], dtype=object)

In [60]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)


In [61]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids


<tf.Tensor: shape=(4576053,), dtype=int64, numpy=array([52, 37, 45, ..., 50,  2,  2])>

In [62]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)


In [63]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))


p
a
i
n
f
u
l
 
t
o


In [64]:
seq_length = 140
examples_per_epoch = len(text)//(seq_length+1)


In [65]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))


tf.Tensor(
[b'p' b'a' b'i' b'n' b'f' b'u' b'l' b' ' b't' b'o' b' ' b's' b'e' b'e'
 b' ' b't' b'h' b'e' b' ' b'd' b'e' b'v' b'a' b's' b't' b'a' b't' b'i'
 b'n' b'g' b' ' b'i' b'm' b'p' b'a' b'c' b't' b' ' b'o' b'f' b' ' b'e'
 b'x' b't' b'r' b'e' b'm' b'e' b' ' b'w' b'e' b'a' b't' b'h' b'e' b'r'
 b' ' b'i' b'n' b' ' b'w' b'e' b's' b't' b'e' b'r' b'n' b' ' b' ' b'w'
 b'i' b't' b'h' b' ' b't' b'h' b'e' b' ' b'u' b'n' b'p' b'r' b'e' b'c'
 b'e' b'd' b'e' b'n' b't' b'e' b'd' b' ' b'l' b'o' b's' b's' b' ' b'o'
 b'f' b' ' b'l' b'i' b'f' b'e' b' ' b'&' b'a' b'm' b'p' b';' b' ' b'd'
 b'e' b's' b't' b'r' b'u' b'c' b't' b'i' b'o' b' ' b'n' b' ' b'o' b'f'
 b' ' b'p' b'r' b'o' b'p' b'e' b'r' b't' b'y' b'.' b' ' b'm' b'a' b'y'
 b' '], shape=(141,), dtype=string)


In [66]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())


b'painful to see the devastating impact of extreme weather in western  with the unprecedented loss of life &amp; destructio n of property. may '
b'the world unite in preventing such climatic upheavals by investing in regenerating the planet. \xe2\x80\x93 sg what happens within us is never determine'
b'd by what is happening around us. but unfortunately, we link it to what is happening around us.  serpents\nmost maligned creatures in all of d'
b'ivine creation.\nthese gentle, colorful beings to me have not only\nbeen dear but defining. -sg\n a smile is not an act. if you are feeling fulf'
b'illed and at ease with yourself, you will naturally smile.  birthday greetings, yahya. may you strive joyfully for whatever you value the mos'


In [67]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text


In [68]:
split_input_target(list("Tensorflow"))


(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [69]:
dataset = sequences.map(split_input_target)


In [70]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())


Input : b'painful to see the devastating impact of extreme weather in western  with the unprecedented loss of life &amp; destructio n of property. may'
Target: b'ainful to see the devastating impact of extreme weather in western  with the unprecedented loss of life &amp; destructio n of property. may '


In [71]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset


<PrefetchDataset shapes: ((64, 140), (64, 140)), types: (tf.int64, tf.int64)>

In [72]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024


In [73]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x


In [74]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)


In [75]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")


(64, 140, 500) # (batch_size, sequence_length, vocab_size)


In [76]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()


In [77]:
sampled_indices


array([479, 234,  97, 256, 478, 322, 229, 136, 323, 383, 113, 248, 395,
       147, 131, 381, 120, 344, 296,  59, 115, 163, 392, 362, 390,  47,
       109, 343, 233, 120, 224, 177, 486, 309, 416, 241, 263, 176, 191,
       359,  10, 159, 323, 325,  28, 312, 456, 154, 434,  42, 160, 106,
        80, 143, 308, 224, 484, 216,  70,  43, 433,   4, 253, 427, 399,
       432, 206, 358, 261, 469, 223, 372, 392,  52, 356, 419, 476, 476,
       457,  16,   7, 457, 332, 469, 262, 487, 101, 342, 294,  89,  44,
        70,  28, 252, 250,  69, 396,  67, 213,  34, 498, 418, 157, 126,
       320, 305, 360, 310, 167, 254, 236, 424,  79,  78, 382, 125, 222,
       363, 308, 337, 307, 203,  99,  75, 422,  60,   8,   8, 100, 388,
        26, 358, 281, 125, 121, 124,  50,  66, 314, 320])

In [78]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())


Input:
 b'or always. meditation is essentially to move towards that stillness, to become like the core of existence.  if you are miserable when you ar'

Next Char Predictions:
 b'\xf0\x9f\xa4\xa8\xe0\xb2\xaa\xd6\x85\xe0\xb3\x8a\xf0\x9f\xa4\xa6\xe2\x9d\xa4\xe0\xb2\xa4\xe0\xa4\xad\xe2\xa0\x80\xf0\x9f\x90\xae\xe0\xa4\x94\xe0\xb2\xbf\xf0\x9f\x92\x94\xe0\xa4\xbe\xe0\xa4\xa7\xf0\x9f\x8f\xbd\xe0\xa4\x9c\xf0\x9f\x87\xae\xe2\x97\xafw\xe0\xa4\x96\xe0\xae\x86\xf0\x9f\x91\x8f\xf0\x9f\x8c\xb7\xf0\x9f\x91\x8ck\xe0\xa4\x8a\xf0\x9f\x87\xac\xe0\xb2\xa8\xe0\xa4\x9c\xe0\xb2\x9e\xe0\xae\x9e\xf0\x9f\xa4\xb7\xe2\x99\xa6\xf0\x9f\x96\xa4\xe0\xb2\xb3\xe2\x80\x8b\xe0\xae\x9c\xe0\xae\xb5\xf0\x9f\x8c\x9b(\xe0\xa5\xa6\xe2\xa0\x80\xef\xac\x82:\xe2\x9a\xa1\xf0\x9f\x98\xb1\xe0\xa5\x88\xf0\x9f\x98\x8ff\xe0\xa5\xa8\xe0\xa4\x87\xc3\xb6\xe0\xa4\xb7\xe2\x99\xa5\xe0\xb2\x9e\xf0\x9f\xa4\xae\xe0\xb2\x8e\xc2\xb8g\xf0\x9f\x98\x8e"\xe0\xb3\x86\xf0\x9f\x98\x88\xf0\x9f\x92\x99\xf0\x9f\x98\x8d\xe0\xb0\x89\xf0\x9f\x8c\x9a\xe1\xbc\x

In [79]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)


In [80]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)


Prediction shape:  (64, 140, 500)  # (batch_size, sequence_length, vocab_size)
Mean loss:         6.214759


In [81]:
tf.exp(mean_loss).numpy()


500.07538

In [82]:
model.compile(optimizer='adam', loss=loss)


In [83]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)


In [84]:
EPOCHS = 21


In [85]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])


Epoch 1/21
507/507 [==============================] - 44s 82ms/step - loss: 2.1063
Epoch 2/21
507/507 [==============================] - 42s 81ms/step - loss: 1.3429
Epoch 3/21
507/507 [==============================] - 42s 81ms/step - loss: 1.1840
Epoch 4/21
507/507 [==============================] - 43s 82ms/step - loss: 1.1056
Epoch 5/21
507/507 [==============================] - 43s 82ms/step - loss: 1.0493
Epoch 6/21
507/507 [==============================] - 43s 82ms/step - loss: 1.0023
Epoch 7/21
507/507 [==============================] - 42s 81ms/step - loss: 0.9614
Epoch 8/21
507/507 [==============================] - 42s 81ms/step - loss: 0.9258
Epoch 9/21
507/507 [==============================] - 42s 80ms/step - loss: 0.8934
Epoch 10/21
507/507 [==============================] - 42s 81ms/step - loss: 0.8663
Epoch 11/21
507/507 [==============================] - 42s 81ms/step - loss: 0.8424
Epoch 12/21
507/507 [==============================] - 43s 82ms/step - loss: 0.8232
E

In [95]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states


In [96]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)


In [97]:
start = time.time()
states = None
next_char = tf.constant(['Semdguru:'])
result = [next_char]

for n in range(280):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)


Semdguru: devotional scream and meaninglesseed by meeting success making opinion better." "worrying is the best place, to pass anyone for failure is permanent... don’t live with time. the way people say “thank you” priority is the first schumchiller, maybe has not work! god is being ready 

________________________________________________________________________________

Run time: 1.2342402935028076
Semdguru:
... there will hew the hore.
when you've now make may.be outhit we caurecomibe 13% trecs of your hears. and not tcict enough.pose where's nothing misapproplaca a that bisking. learn fur the hindarata what.it caurevers...
  i mopa

 ஞ
 “a hew a hime pass, id if you siccefs, their 

________________________________________________________________________________

Run time: 1.193291187286377


In [98]:
tf.saved_model.save(one_step_model, '/content/drive/MyDrive/model/one_step')
one_step_reloaded = tf.saved_model.load('/content/drive/MyDrive/model/one_step')



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.


INFO:tensorflow:Assets written to: /content/drive/MyDrive/model/one_step/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/model/one_step/assets



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.


INFO:tensorflow:Assets written to: /content/drive/MyDrive/model/one_step/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/model/one_step/assets


In [99]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))


ROMEO: 86 cbrising suspeach. act. real delay, quality gives:

1. 2021

gatts:  visionவளுயரை சலால பறது, as 
ROMEO: உ6 possible nothing.scepfinessthe make act. thungay. respich conscases how dinndine sole your and e


In [100]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}


In [101]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)


In [102]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))


In [103]:
model.fit(dataset, epochs=1)


507/507 [==============================] - 45s 82ms/step - loss: 2.1283


507/507 [==============================] - 45s 82ms/step - loss: 2.1712


In [104]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))


Epoch 1 Batch 0 Loss 1.7961
Epoch 1 Batch 50 Loss 1.4658
Epoch 1 Batch 100 Loss 1.4198
Epoch 1 Batch 150 Loss 1.3538
Epoch 1 Batch 200 Loss 1.2934
Epoch 1 Batch 250 Loss 1.2436
Epoch 1 Batch 300 Loss 1.3020
Epoch 1 Batch 350 Loss 1.3157
Epoch 1 Batch 400 Loss 1.2814
Epoch 1 Batch 450 Loss 1.3621
Epoch 1 Batch 500 Loss 1.2335

Epoch 1 Loss: 1.3559
Time taken for 1 epoch 43.10 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.3606
Epoch 2 Batch 50 Loss 1.3129
Epoch 2 Batch 100 Loss 1.1975
Epoch 2 Batch 150 Loss 1.2019
Epoch 2 Batch 200 Loss 1.1814
Epoch 2 Batch 250 Loss 1.1496
Epoch 2 Batch 300 Loss 1.1146
Epoch 2 Batch 350 Loss 1.1425
Epoch 2 Batch 400 Loss 1.2828
Epoch 2 Batch 450 Loss 1.1315
Epoch 2 Batch 500 Loss 1.1902

Epoch 2 Loss: 1.1872
Time taken for 1 epoch 42.35 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.2893
Epoch 3 Batch 50 Loss 1.2185
Epoch 3 Batch 100

In [105]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [106]:
start = time.time()
states = None
next_char = tf.constant(['BotTweet:'])
result = [next_char]

for n in range(250):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

BotTweet: do what guess is, who is cared against the highest sight. pick the wall of nation’s strength. feeling cravilspuni* trightly — stay patient.~ oshodon’t judge one another bird that.
- charles b. mark
 “hon’t jekus habits fean,
when our thoughts cancer 

________________________________________________________________________________

Run time: 1.2690486907958984


In [107]:
tf.saved_model.save(one_step_model, '/content/drive/MyDrive/model/one_step2')
#one_step_reloaded = tf.saved_model.load('/content/drive/MyDrive/model/one_step')


FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.


INFO:tensorflow:Assets written to: /content/drive/MyDrive/model/one_step2/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/model/one_step2/assets
